In [486]:
import os
import nltk   
import re
import string
import operator

from gensim import corpora
from collections import Counter
from math import *

In [353]:
#ls

In [354]:
os.chdir('/Users/andrewvangilder/Desktop/Python/Rick_And_Morty_Scripts')

In [355]:
scripts = []
for root, dirs, files in os.walk('.'):
    for file in files:
        if file.endswith('.txt'):
            scripts.append(file)

In [356]:
#scripts

In [357]:
chunky_text = []
chunky_text_lines = []

for i in range(0,len(scripts)):
    with open(scripts[i]) as doc:
        chunky_text.append(nltk.word_tokenize(doc.read()))
        
        
for i in range(0,len(scripts)):
    with open(scripts[i]) as doc:
        chunky_text_lines.append(''.join(doc.readlines()))
        
#print(chunky_text_lines[0])

In [367]:
#Finding a way to return list of each characters' lines
#
#

morty_lines = []
rick_lines = []
summer_lines = []
jerry_lines = []

morty_words = []
rick_words = []
summer_words = []
jerry_words = []

for i in range(0,len(chunky_text_lines)):
    for match in re.finditer("Morty:"+".*", chunky_text_lines[i]):
        morty_lines.append(chunky_text_lines[i][int(match.start()):int(match.end())])

    for match in re.finditer("Rick:"+".*", chunky_text_lines[i]):
        rick_lines.append(chunky_text_lines[0][int(match.start()):int(match.end())])

    for match in re.finditer("Summer:"+".*", chunky_text_lines[i]):
        summer_lines.append(chunky_text_lines[i][int(match.start()):int(match.end())])
        
    for match in re.finditer("Jerry:"+".*", chunky_text_lines[i]):
        jerry_lines.append(chunky_text_lines[i][int(match.start()):int(match.end())])


#clean out the stage directions, found in parentheses 
#Also make list of words each character uses 
for i in range(0,len(morty_lines)):
    morty_lines[i] = re.sub("\((?<=\().*(?=\))"+"\)",'',morty_lines[i])
    morty_words.extend(re.split(r'[\*\?\.\,!" "]', morty_lines[i]))
for i in range(0,len(rick_lines)):
    rick_lines[i] = re.sub("\((?<=\().*(?=\))"+"\)",'',rick_lines[i])
    rick_lines[i] = re.sub("\\n",'',rick_lines[i])
    rick_words.extend(re.split(r'[\*\?\.\,!" "]', rick_lines[i]))
for i in range(0,len(summer_lines)):
    summer_lines[i] = re.sub("\((?<=\().*(?=\))"+"\)",'',summer_lines[i])
    summer_words.extend(re.split(r'[\*\?\.\,!" "]', summer_lines[i]))
for i in range(0,len(jerry_lines)):
    jerry_lines[i] = re.sub("\((?<=\().*(?=\))"+"\)",'',jerry_lines[i])
    jerry_words.extend(re.split(r'[\*\?\.\,!" "]', jerry_lines[i]))


In [385]:
#list of words to clean out of characters' vocabulary

#words_to_clean = list(string.punctuation) + list(string.ascii_lowercase) + characters + nltk.corpus.stopwords.words("english") + [ "'ve",'‘','”','“','``',"''",'’','\'s','n\'t','\'re','\'m','--','...','1']

words_to_clean = ['','morty:','rick:','summer:','jerry:','[',']']

In [386]:
morty_words = [word.lower() for word in morty_words if word.lower() not in words_to_clean]
rick_words = [word.lower() for word in rick_words if word.lower() not in words_to_clean]
summer_words = [word.lower() for word in summer_words if word.lower() not in words_to_clean]
jerry_words = [word.lower() for word in jerry_words if word.lower() not in words_to_clean]

In [453]:
morty_counter = Counter(morty_words)
rick_counter = Counter(rick_words)
summer_counter = Counter(summer_words)
jerry_counter = Counter(jerry_words)

#jerry_counter.most_common()[:50]

$\fract{1+log(f_{t,d})log\frac{N}{n_{term}}}{1+log(f_{t,c})log\frac{N}{n_{term}}}$

In [454]:
morty_dictionary = dict(morty_counter)
rick_dictionary = dict(rick_counter)
summer_dictionary = dict(summer_counter)
jerry_dictionary = dict(jerry_counter)

#morty_counter.most_common()[:50]

In [455]:
morty_dictionary['go']

17

In [473]:
#functions to return relative term frequency (tf)

def tf(dictionary,term):
    corpus_len = 0
    
    for key in dictionary:
        corpus_len += dictionary[key]
    return dictionary[term]/corpus_len
    
def tf_dict(dictionary):
    corpus_len = 0
    tf_dict = {}
    
    for key in dictionary:
        corpus_len += dictionary[key]
    for key in dictionary:
        tf_dict[key] = dictionary[key]/corpus_len
        
    return tf_dict

In [522]:
#function to return smoothed inverse document frequency (smoothed idf)

def idf(word,*args):
    
    dictionary_list = []
    doc_count = 0
    
    for arg in args:
        dictionary_list.append(arg)
        if word in arg:
            doc_count += 1
    
    return log((1.1+doc_count)/len(dictionary_list))


In [523]:
def tf_idf_dicts(*args):
    
    tf_idf_dict_list = []
    
    for arg in args:
        tf_idf_dict = {}
        for key in tf_dict(arg):
            tf_idf_dict[key] = tf_dict(arg)[key]*idf(key,*args)
        tf_idf_dict_list.append(tf_idf_dict)
        
    return tf_idf_dict_list
            
    

In [524]:
tf_idf_dict_list = tf_idf_dicts(morty_dictionary,rick_dictionary,summer_dictionary,jerry_dictionary)

In [525]:
#tf_dict(morty_dictionary)[key]

#tf_idf_dict_list[0]

In [526]:
sorted_tf_idf_morty = sorted(tf_idf_dict_list[3].items(), key=operator.itemgetter(1))

sorted_tf_idf_morty[-1000:]

[('toe-up', -0.00017169118475633178),
 ('flow-up', -0.00017169118475633178),
 ('whatever’s', -0.00017169118475633178),
 ('gurgling', -0.00017169118475633178),
 ('noticing', -0.00017169118475633178),
 ('spinning', -0.00017169118475633178),
 ('credit', -0.00017169118475633178),
 ('hidden', -0.00017169118475633178),
 ('subterranean', -0.00017169118475633178),
 ('lair', -0.00017169118475633178),
 ('faceless', -0.00017169118475633178),
 ('gargler', -0.00017169118475633178),
 ('hating', -0.00017169118475633178),
 ('playa', -0.00017169118475633178),
 ('jack', -0.00017169118475633178),
 ('steed', -0.00017169118475633178),
 ('thousand', -0.00017169118475633178),
 ('dollar', -0.00017169118475633178),
 ('electric', -0.00017169118475633178),
 ('bill', -0.00017169118475633178),
 ('prisoners', -0.00017169118475633178),
 ('reeeal-', -0.00017169118475633178),
 ('explain', -0.00017169118475633178),
 ('upstairs', -0.00017169118475633178),
 ('carrots', -0.00017169118475633178),
 ('lifetime', -0.000171691

In [496]:
#idf('ricks',rick_dictionary,morty_dictionary,jerry_dictionary,summer_dictionary)

In [425]:
morty_corpus_len = 0
tf_dict = {}

for key in morty_dictionary:
    morty_corpus_len += morty_dictionary[key]
for key in morty_dictionary:
    tf_dict[key] = morty_dictionary[key]/morty_corpus_len
    
print(tf(morty_dictionary,'you')/tf_dict['you'])

1.0
